# Projeto 1 - Ciência dos Dados

Nome: Antônio Amaral

Nome: Gabriel Penna

Nome: Caio Garcia

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
#Definindo Função Magica iPython
%matplotlib inline
#Importanto Pandas
import pandas as pd
#Importando MatPlotLib e Numpy
import matplotlib.pyplot as plt
import numpy as np
#Importando Sistema Operacional
import os
#Importando RE
import re

In [3]:
#Checando Diretório que será trabalhado:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/antonioamaralegydiomartins/Desktop/Antônio/GitHub - Pessoal/INSPER/CDados_Projeto1/Etapa 1 e 2


## Formulas Utilizadas:

In [4]:
def laplace (frase,tabela_rel, tabela_irrel,nao_repetentes,len_rel,len_irrel):
    #Definindo lista de probabilidades
    probR_list=[]
    probIr_list=[]
    #Separando a Frase em elementos
    splt = frase.split()
    #For para reconhecer a quantidade 
    for palavra in splt:
        if palavra not in tabela_rel:
            qtR = 1
        else:
            qtR = tabela_rel[palavra] + 1
        if palavra not in tabela_irrel:
            qtIr = 1
        else:
            qtIr = tabela_irrel[palavra] + 1
        #Definindo divisores
        div1 = len_rel + len(nao_repetentes)
        div2 = len_irrel + len(nao_repetentes)
        #CCalculo de Laplace
        probR_list.append(qtR/div1)
        probIr_list.append(qtIr/div2)
    return (probR_list,probIr_list)

def cleanup(texto):

    temp = texto.lower()

    temp = re.sub("@[A-Za-z0-9_]+","", temp)

    temp = re.sub("#[A-Za-z0-9_]+","", temp)

    temp = re.sub(r'http\S+', '', temp)

    temp = re.sub('[()!?ㅋ;:,/^~♂️*=&-]', ' ', temp)

    temp = re.sub('\[.*?\]',' ', temp)

    temp = re.sub("[z0-9]"," ", temp)

    temp = re.sub('\\\\\\\\n'," ", temp)

    temp = re.sub('\\\\'," ", temp)

    temp = re.sub('\ '," ", temp)
    
    temp = re.sub("é","e",temp)
    
    temp = re.sub("ã","a",temp)

    return temp

## Carregando Arquivos

In [5]:
#Definindo Arquivo que será trabalhado
filename = 'yakult-final.xlsx'

In [6]:
#Carregando Excel dentro da Variável Trainamento
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,@colchoesueli a médica passou lactobacilos p...,1.0
1,alguém compra yakult pra mim por deus,0.0
2,"amanhã vou fazer aquele drink de morango, ...",0.0
3,tomando um yakult no meio da rua 💪,1.0
4,48 horas e estou sobrevivendo com;\\\\nmei...,1.0


In [7]:
#Carregando Excel dentro da Variável Teste
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,"@otavio_amaral18 é uma delícia juro, desce...",1
1,"o cérebro é muito impressionante, ontem eu...",1
2,@fenicolli02 quem q pede gole do yakult???...,0
3,@thamaramaia5 @adrianommartins @taaayrd @t...,0
4,sdds das mamis juntas brindando yakult @fe...,0


## Treinamento Naive-Bayles --- Classificador Automático de Sentimento

Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto que foi escolhido para ser analizado foi o yakult, a famosa bebida fermentada japonesa, o grupo levou em consideração a emissão de opinião na hora de montar o seu classificador. Sendo assim se houve a emissão de uma uma opinião, sendo benéfica ou não, o Tweet é considerado relevante, porém se houver a emissão de uma opinião que não corresponde ao produto, ou não houve a emissão de uma opinião o Tweet é considerado irrelevante.

___
### - Definindo Relevância 

In [8]:
#Definindo quais são os Tweets com classificação Irrelevante, dentro da Sheet Treinamento
Irrelevantes=train.loc[train['Classificação']==0,'Treinamento']
Irrelevantes

1                alguém compra yakult pra mim por deus  
2          amanhã vou fazer aquele drink de morango, ...
5          @madrugatina sei que vc não bebe, então va...
6          meu lindo :( quero ver ele fazendo batida ...
7          nadei igual um desgraçado, agora comi bata...
                             ...                        
482        talvez eu faça aquela bebida de morango co...
486        meu maior sonho atualmente é um yakult veg...
492        @bcamoezi esses dias comprei couve, alface...
495        ontem eu comi...\\\\npão na chapa\\\\nstak...
497        ontem eu cheguei de madrugada em casa e fi...
Name: Treinamento, Length: 285, dtype: object

In [9]:
#Definindo quais são os Tweets com classificação Relevante, dentro da Sheet Treinamento
Relevantes=train.loc[train['Classificação']==1,'Treinamento']
Relevantes

0        @colchoesueli a médica passou lactobacilos p...
3                   tomando um yakult no meio da rua 💪  
4          48 horas e estou sobrevivendo com;\\\\nmei...
14         minha sogra fala mais que a mulher do yaku...
15         última coisa q comeu — tomei um yakult de ...
                             ...                        
490        a garrafinha de yakult é mt pequena, mano,...
491        @caarolmario bebida de yakult com corote (...
493        acabei de ingerir isso aqui, me disseram q...
494        bernardo aprendeu a tomar yakult de canudi...
496      bolo: red velvet ou chocolate\\\\ndoce: chee...
Name: Treinamento, Length: 213, dtype: object

___
### - Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Criando variáveis e realizando limpeza:

In [10]:
#Definindo texto_relevante, como a junção de todos os Tweets relevantes
texto_relevante=' '.join(Relevantes)

In [11]:
#Definindo texto_irrelevante como a junção de todos os Tweets irrelevantes
texto_irrelevante=' '.join(Irrelevantes)

In [12]:
#Passando ambos os textos pela função Cleanup, e convertendo todas as letras para Lowercase
texto_limpo_relevante=cleanup(texto_relevante).lower()
texto_limpo_irrelevante=cleanup(texto_irrelevante).lower()

Convertendo as strings limpas em objetos do tipo pd.series:

In [13]:
palavras_relevantes = texto_limpo_relevante.split()
serie_rel = pd.Series(palavras_relevantes)

In [14]:
palavras_irrelevantes = texto_limpo_irrelevante.split()
serie_irrel = pd.Series(palavras_irrelevantes)

Tornando os objetos transformados acima em series relativas:

In [15]:
serie_irel = serie_rel.value_counts(True)

In [16]:
serie_rrel = serie_irrel.value_counts(True)

In [17]:
#Checando porcentagem das listas
print("Porcentagem de Irrelevantes: {0} e de Relevantes: {1}".format(serie_irel.sum(),serie_rrel.sum()))

Porcentagem de Irrelevantes: 1.0 e de Relevantes: 1.0000000000000002


Montando serie com todas as palavras que aparecem em todos os Tweets:

In [18]:
tabela_rel = serie_rel.value_counts()

In [19]:
tabela_irrel = serie_irrel.value_counts()

In [20]:
tabela_rel_rel = serie_rel.value_counts(True)

In [21]:
tabela_rel_irrel = serie_irrel.value_counts(True)

In [22]:
#Somando Palavras Relevantes e Irrelevantes para criar um dicionário de palavras
todas = texto_limpo_relevante + texto_limpo_irrelevante
#Criando serie_total
total_palavras = todas.split()
serie_total = pd.Series(total_palavras)
#Criando a tabela total de palavras
tabela_total = serie_total.value_counts()

In [23]:
print("Procentagem da Tabela de Irrelevantes: {0} e de Relevantes: {1}".format(tabela_irrel.sum(),tabela_rel.sum()))

Procentagem da Tabela de Irrelevantes: 4222 e de Relevantes: 3230


Adicionando variáveis que representam a soma das tabelas irrel e rel:

In [24]:
RelPal = tabela_rel.sum()


In [25]:
IrrelPal = tabela_irrel.sum()

In [26]:
TotalPal = tabela_total.sum()

Descobrindo a Probabilidade de ser uma palavra Relevante e Irrelevante:

In [27]:

probI=IrrelPal/TotalPal
print("A probabilidade de uma palavra ser Irrelevante: {0}".format(probI))

A probabilidade de uma palavra ser Irrelevante: 0.5665593129361245


In [28]:
probR=RelPal/TotalPal
print("A probabilidade de uma palavra ser Relevante: {0}".format(probR))

A probabilidade de uma palavra ser Relevante: 0.43344068706387545


In [29]:
lista_tabela_total = serie_total.tolist()
nao_repetentes = set(lista_tabela_total)
nao_repetentes = pd.DataFrame(nao_repetentes)

In [30]:
Frase = "gosto muito estou viciado abri geladeira"
Frase = cleanup(Frase).lower()
Frase

'gosto muito estou viciado abri geladeira'

In [31]:
len_rel = len(tabela_rel)
len_irrel = len(tabela_irrel)

In [32]:
probfR = laplace(Frase, tabela_rel, tabela_irrel, nao_repetentes, len_rel, len_irrel)[0]
probfIr = laplace(Frase, tabela_rel, tabela_irrel, nao_repetentes, len_rel, len_irrel)[1]
 
pR = 1
pIr = 1

for e in range(len(probfR)):
    pR = pR * probfR[e]
    pIr = pIr * probfIr[e]

In [33]:
if pR > pIr:
    print("Relevante")
else:
    print("Irrelevante")

Relevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [52]:
list_prob = []
diver_rel = []
Tweets = []
for e in test["Teste"]:
    text_clean_test = cleanup(e).lower()
    Tweets.append(text_clean_test)
    
List_Relevancia = []
for Tweet in Tweets:
    probfR2 = laplace(Tweet, tabela_rel, tabela_irrel, nao_repetentes, len_rel, len_irrel)[0]
    probfIr2 = laplace(Tweet, tabela_rel, tabela_irrel, nao_repetentes, len_rel, len_irrel)[1]
    pR2 = 1
    pIr2 = 1
    for e in range(len(probfR2)):
        pR2 = pR2 * probfR2[e]
        pIr2 = pIr2 * probfIr2[e]
    if pR2 > pIr2:
        list_prob.append(pR2)
        List_Relevancia.append(1)
        val = pIr2/pR2
        if val > 0.6:
            diver_rel.append(4)
        elif val >0.3 and val <=0.6:
            diver_rel.append(3)
        elif val >0 and val <=0.3:
            diver_rel.append(2)
    else:
        list_prob.append(pIr2)
        List_Relevancia.append(0)
        val = pR2/pIr2
        if val > 0.6:
            diver_rel.append(0)
        elif val >0.3 and val <=0.6:
            diver_rel.append(1)
        elif val >0 and val <=0.3:
            diver_rel.append(2)
    
test["Classificador"] = List_Relevancia
test["Probabilidade"] = list_prob
test["Multi Classificação"] = diver_rel

,Teste,Classificação,Classificador,Probabilidade,Multi Classificação
0,"@otavio_amaral18 é uma delícia juro, desce...",1,1,2.276085e-22,2
1,"o cérebro é muito impressionante, ontem eu...",1,0,4.684975e-121,1
2,@fenicolli02 quem q pede gole do yakult???...,0,0,1.122744e-13,2
3,@thamaramaia5 @adrianommartins @taaayrd @t...,0,0,1.019671e-07,1
4,sdds das mamis juntas brindando yakult @fe...,0,0,1.822636e-18,0
...,...,...,...,...,...
315,7 da manhã eu e belle tomando yakult e comend...,1,0,8.276394e-31,2
316,não falei nada ai vem uma chata me responder ...,1,1,1.056109e-55,2
317,@01001010julio @acecourtbot conseguiu sua merda,0,0,2.873098e-10,0
318,"falei pro meu irmão q tava doentinha, pedi pr...",1,1,2.357538e-95,2


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**